# 📘 Introducción

## ¿Qué es el polipropileno?

El **polipropileno (PP)** es un polímero termoplástico perteneciente a la familia de las poliolefinas. Se obtiene mediante la **polimerización del propileno**, un derivado del petróleo.  
Es uno de los plásticos más utilizados a nivel mundial debido a su bajo costo, versatilidad, ligereza y resistencia química.

Entre sus principales aplicaciones destacan:

- Empaques y envases  
- Industria automotriz  
- Textiles y fibras  
- Construcción  
- Dispositivos médicos  
- Productos del hogar  

Su combinación de precio accesible, resistencia y facilidad de moldeo lo convierten en un material indispensable en la manufactura moderna.

---

## ¿Por qué su precio es relevante?

El precio del polipropileno es un indicador económico importante, pues está influenciado por factores interconectados como:

- Precios del petróleo y gas natural  
- Oferta y demanda global  
- Situación de la industria petroquímica  
- Políticas comerciales y condiciones macroeconómicas  
- Paros técnicos o fallas en plantas de producción  

Su relevancia se debe a que el PP afecta directamente los **costos de producción** de múltiples industrias.  
Por ello, contar con **predicciones confiables del precio del PP** ayuda a:

- Planificar compras y presupuestos  
- Evaluar riesgos de precios  
- Estimar márgenes  
- Realizar análisis económicos de mercado  

---

## Breve revisión del dataset

Los datos fueron obtenidos mediante un método de scraping controlado desde TradingEconomics, tomando los valores históricos del precio spot del polipropileno.

El dataset contiene dos columnas principales:

- **Date** — Fecha de registro  
- **Price** — Precio del polipropileno por tonelada  

Ejemplo de datos:

| Date       | Price |
|------------|-------|
| 2024-11-20 | 7489  |
| 2024-11-21 | 7510  |
| 2024-11-22 | 7504  |
| 2024-11-25 | 7467  |
| 2024-11-26 | 7479  |

Características del dataset:

- Frecuencia: **Diaria**  
- Formato: **serie temporal**  
- Moneda: **USD por tonelada**  
- Datos limpios y sin valores nulos  
- Ordenados de forma cronológica  



---


In [ ]:
#importar datos desde tampermokey
import pandas as pd
import requests

df = pd.DataFrame(requests.get("http://localhost:8765/pp").json())

df = df.rename(columns={"date": "Date", "value": "Price"})
df["Date"] = pd.to_datetime(df["Date"])
df.head()


,Date,Price
0,2024-11-21,7510
1,2024-11-22,7504
2,2024-11-25,7467
3,2024-11-26,7479
4,2024-11-27,7481


In [28]:
import pandas as pd
from pandas import read_csv 
df = pd.DataFrame(read_csv("commodity_data.csv"))
df = df.rename(columns={"date": "Date", "value": "Price"})
df["Date"] = pd.to_datetime(df["Date"])
df.head()

,Date,Price
0,2024-11-21,7510
1,2024-11-22,7504
2,2024-11-25,7467
3,2024-11-26,7479
4,2024-11-27,7481


In [21]:
import pandas as pd
import plotly.express as px



fig = px.line(
    df,
    x="Date",
    y="Price",
    title="Precio del Polipropileno (PP) — Histórico",
    markers=True
)

fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Precio (CNY/T)",
    hovermode="x unified",
    template="plotly_dark",
    title_font=dict(size=22),
    height=500
)



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Date=%{x}<br>Price=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2024-11-21T00:00:00.000000000', '2024-11-22T00:00:00.000000000',
                          '2024-11-25T00:00:00.000000000', ..., '2025-11-17T00:00:00.000000000',
                          '2025-11-18T00:00:00.000000000', '2025-11-19T00:00:00.000000000'],
                         shape=(241,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('Vh1QHSsdNx05HTodMx0zHS8dJh0ZHS' ... 'MZCxkbGRoZFhkLGQcZWhlBGRsZFhk='),
                    'dtype': 'i2'},
              'yaxis': 'y'}],
    'layout': {'height': 500,
               'hovermode': 'x unified',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'font': {'size': 22}, 'text': 'Precio del Polipropileno (PP) — Histórico'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Fecha'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Precio (CNY/T)'}}}
})

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["Price_scaled"] = scaler.fit_transform(df[["Price"]])
df.head()


,Date,Price,Price_scaled
0,2024-11-20,7489,0.980961
1,2024-11-21,7510,1.000000
2,2024-11-22,7504,0.994560
3,2024-11-25,7467,0.961015
4,2024-11-26,7479,0.971895


In [10]:
import numpy as np

data = df["Price_scaled"].values

train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data  = data[train_size:]


In [ ]:
def create_windows(series, window_size=30):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)


In [ ]:
WINDOW = 30  # puedes cambiarlo después

X_train, y_train = create_windows(train_data, WINDOW)
X_test,  y_test  = create_windows(test_data,  WINDOW)

# Ajustar dimensiones para LSTM/CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(WINDOW, 1)),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


ModuleNotFoundError: No module named 'tensorflow'